In [3]:
from __future__ import annotations
import heapq
import random
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Set
import math

try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except Exception:
    HAS_MATPLOTLIB = False

def now_str(t: float) -> str:
    return f"{t:.3f}s"

class Event:
    def __init__(self, time: float, target: Any, action: Callable, description: str = ""):
        self.time = time
        self.target = target
        self.action = action
        self.description = description
    def __lt__(self, other: "Event"):
        return self.time < other.time

class Message:
    def __init__(self, src: str, dst: str, kind: str, payload: Any = None, msg_id: Optional[int] = None):
        self.src = src
        self.dst = dst
        self.kind = kind
        self.payload = payload
        self.msg_id = msg_id if msg_id is not None else id(self)
    def __repr__(self):
        return f"Message({self.kind} {self.src}->{self.dst} id={self.msg_id})"

class Node:
    def __init__(self, node_id: str, simulator: "Simulator"):
        self.node_id = node_id
        self.sim = simulator
        self.alive = True
        self.app_state: Dict[str, Any] = {}
        self.received: List[Tuple[float, Message]] = []
        self.sent_count = 0
        self.received_count = 0
    def log(self, msg: str):
        self.sim.log(f"[{self.node_id} @ {now_str(self.sim.time)}] {msg}")
    def send(self, dst: str, kind: str, payload: Any = None, delay: Optional[float] = None):
        if not self.alive:
            self.log("cannot send: node is crashed")
            return
        message = Message(self.node_id, dst, kind, payload)
        self.sent_count += 1
        self.sim.network.send(message, delay)
    def broadcast(self, kind: str, payload: Any = None):
        for other in self.sim.network.nodes():
            if other != self.node_id:
                self.send(other, kind, payload)
    def on_receive(self, msg: Message):
        self.received.append((self.sim.time, msg))
        self.received_count += 1
        self.log(f"received {msg}")
    def crash(self):
        self.alive = False
        self.log("CRASHED")
    def recover(self):
        self.alive = True
        self.log("RECOVERED")

class Network:
    def __init__(self, simulator: "Simulator", base_delay: float = 1.0, jitter: float = 0.5, loss_prob: float = 0.0):
        self.sim = simulator
        self._nodes: Dict[str, Node] = {}
        self.base_delay = base_delay
        self.jitter = jitter
        self.loss_prob = loss_prob
        self.partitions: Dict[str, int] = {}
    def add_node(self, node: Node):
        self._nodes[node.node_id] = node
    def nodes(self) -> List[str]:
        return list(self._nodes.keys())
    def set_partition(self, node: str, partition_id: Optional[int]):
        if partition_id is None:
            if node in self.partitions:
                del self.partitions[node]
        else:
            self.partitions[node] = partition_id
    def same_partition(self, a: str, b: str) -> bool:
        pa = self.partitions.get(a, None)
        pb = self.partitions.get(b, None)
        return pa == pb
    def sample_delay(self) -> float:
        return max(0.0, random.gauss(self.base_delay, self.jitter))
    def send(self, message: Message, override_delay: Optional[float] = None):
        if random.random() < self.loss_prob:
            self.sim.log(f"NETWORK: message lost {message}")
            return
        if not self.same_partition(message.src, message.dst):
            self.sim.log(f"NETWORK: partition drop {message}")
            return
        delay = override_delay if override_delay is not None else self.sample_delay()
        deliver_time = self.sim.time + delay
        def deliver(sim: Simulator, ev: Event):
            dst_node = self._nodes.get(message.dst, None)
            if dst_node is None:
                sim.log(f"DELIVER FAIL: unknown dst {message.dst}")
                return
            if not dst_node.alive:
                sim.log(f"DELIVER: node {dst_node.node_id} down, message lost {message}")
                return
            dst_node.on_receive(message)
        ev = Event(deliver_time, message.dst, deliver, description=f"deliver {message}")
        self.sim.schedule(ev)
        self.sim.log(f"NETWORK: scheduled {message} -> {message.dst} at {now_str(deliver_time)}")

class Simulator:
    def __init__(self, until: float = 100.0, random_seed: Optional[int] = None, verbose: bool = True):
        self.time = 0.0
        self.queue: List[Event] = []
        self.network = Network(self)
        self.nodes: Dict[str, Node] = {}
        self.until = until
        self.verbose = verbose
        self.random_seed = random_seed
        if random_seed is not None:
            random.seed(random_seed)
        self._logs: List[str] = []
    def log(self, msg: str):
        text = f"[{now_str(self.time)}] {msg}"
        self._logs.append(text)
        if self.verbose:
            print(text)
    def schedule(self, ev: Event):
        heapq.heappush(self.queue, ev)
    def add_node(self, node_id: str) -> Node:
        node = Node(node_id, self)
        self.nodes[node_id] = node
        self.network.add_node(node)
        self.log(f"node added: {node_id}")
        return node
    def crash_node(self, node_id: str):
        node = self.nodes.get(node_id)
        if node:
            node.crash()
    def recover_node(self, node_id: str):
        node = self.nodes.get(node_id)
        if node:
            node.recover()
    def partition(self, partition_map: Dict[str, int]):
        for n,p in partition_map.items():
            self.network.set_partition(n, p)
        self.log(f"network partitions set: {partition_map}")
    def heal_partitions(self):
        for n in list(self.network.partitions.keys()):
            self.network.set_partition(n, None)
        self.log("network partitions healed")
    def run(self):
        self.log(f"SIM START until={self.until} SDG 4")
        while self.queue and self.time <= self.until:
            ev = heapq.heappop(self.queue)
            if ev.time > self.until:
                break
            self.time = ev.time
            try:
                ev.action(self, ev)
            except Exception as e:
                self.log(f"ERROR in event {ev.description}: {e}")
        self.log("SIM END")
    def summary(self) -> Dict[str, Any]:
        s = {
            'time': self.time,
            'nodes': {nid: {'alive': n.alive, 'sent': n.sent_count, 'received': n.received_count} for nid,n in self.nodes.items()},
            'logs': self._logs,
        }
        return s

class FailureSchedule:
    def __init__(self, sim: Simulator):
        self.sim = sim
    def crash_after(self, node_id: str, t: float):
        def act(sim: Simulator, ev: Event):
            sim.crash_node(node_id)
        self.sim.schedule(Event(self.sim.time + t, node_id, act, description=f"crash {node_id}"))
    def crash_at(self, node_id: str, t_abs: float):
        def act(sim: Simulator, ev: Event):
            sim.crash_node(node_id)
        self.sim.schedule(Event(t_abs, node_id, act, description=f"crash {node_id}"))
    def recover_at(self, node_id: str, t_abs: float):
        def act(sim: Simulator, ev: Event):
            sim.recover_node(node_id)
        self.sim.schedule(Event(t_abs, node_id, act, description=f"recover {node_id}"))

def demo_scenario():
    sim = Simulator(until=60.0, random_seed=42, verbose=True)
    sim.network.base_delay = 2.0
    sim.network.jitter = 1.0
    sim.network.loss_prob = 0.05
    nodes = [sim.add_node(f"N{i}") for i in range(1,6)]
    def make_gossip_handler(node: Node):
        seen: Set[Any] = set()
        def handler(msg: Message):
            if not node.alive:
                return
            if msg.kind == 'GOSSIP':
                if msg.payload in seen:
                    return
                seen.add(msg.payload)
                node.log(f"gossip received {msg.payload}")
                peers = [p for p in node.sim.network.nodes() if p != node.node_id]
                k = max(1, int(math.log(max(1, len(peers)))))
                targets = random.sample(peers, k)
                for t in targets:
                    node.sim.network.send(Message(node.node_id, t, 'GOSSIP', payload=msg.payload))
        return handler
    for n in nodes:
        n.on_receive = make_gossip_handler(n)
    def start_gossip(sim: Simulator, ev: Event):
        sim.log("starting gossip from N1")
        sim.nodes['N1'].sim.network.send(Message('N1', 'N2', 'GOSSIP', payload='hello'))
    sim.schedule(Event(1.0, 'controller', start_gossip, description='start gossip'))
    fail = FailureSchedule(sim)
    fail.crash_at('N3', 10.0)
    fail.recover_at('N3', 25.0)
    def partition_on(sim: Simulator, ev: Event):
        sim.partition({'N4': 2, 'N5': 2, 'N1': 1, 'N2': 1, 'N3': 1})
    def partition_off(sim: Simulator, ev: Event):
        sim.heal_partitions()
    sim.schedule(Event(15.0, 'controller', partition_on, description='partition on'))
    sim.schedule(Event(35.0, 'controller', partition_off, description='partition off'))
    sim.run()
    return sim

if __name__ == '__main__':
    sim = demo_scenario()
    summary = sim.summary()
    print('\nSUMMARY SDG 4:')
    for nid,info in summary['nodes'].items():
        print(f"{nid}: alive={info['alive']} sent={info['sent']} received={info['received']}")
    print('\nSDG 4 STATUS: Knowledge dissemination')
    for nid, info in summary['nodes'].items():
        status = 'Content received' if info['received'] > 0 else 'No content'
        print(f"{nid}: {status}")
    print('\nThis project supports SDG 4 – Quality Education by:')
    print('1. Demonstrating knowledge dissemination across nodes.')
    print('2. Teaching resilience in learning systems through failure simulations.')
    print('3. Enhancing digital literacy and technical skills via simulation.')
    print('4. Providing a platform for research and training for future-ready learners.')

   

[0.000s] node added: N1
[0.000s] node added: N2
[0.000s] node added: N3
[0.000s] node added: N4
[0.000s] node added: N5
[0.000s] SIM START until=60.0 SDG 4
[1.000s] starting gossip from N1
[1.000s] NETWORK: scheduled Message(GOSSIP N1->N2 id=1407047611600) -> N2 at 3.792s
[3.792s] [N2 @ 3.792s] gossip received hello
[3.792s] NETWORK: scheduled Message(GOSSIP N2->N3 id=1407047611648) -> N3 at 5.918s
[5.918s] [N3 @ 5.918s] gossip received hello
[5.918s] NETWORK: scheduled Message(GOSSIP N3->N1 id=1407047612464) -> N1 at 8.250s
[8.250s] [N1 @ 8.250s] gossip received hello
[8.250s] NETWORK: message lost Message(GOSSIP N1->N5 id=1407047612416)
[10.000s] [N3 @ 10.000s] CRASHED
[15.000s] network partitions set: {'N4': 2, 'N5': 2, 'N1': 1, 'N2': 1, 'N3': 1}
[25.000s] [N3 @ 25.000s] RECOVERED
[35.000s] network partitions healed
[35.000s] SIM END

SUMMARY SDG 4:
N1: alive=True sent=0 received=0
N2: alive=True sent=0 received=0
N3: alive=True sent=0 received=0
N4: alive=True sent=0 received=0
N5: